In [1]:
import pandas as pd
import glob

# 1️⃣ 현재 폴더의 모든 'recipes_over100k_' 로 시작하는 CSV 파일 불러오기
csv_files = glob.glob("recipes_over100k_*.csv")

# 2️⃣ 파일 확인
print("📂 병합할 파일 목록:")
for f in csv_files:
    print(" -", f)

# 3️⃣ 파일 읽어서 하나의 DataFrame으로 합치기
dfs = [pd.read_csv(f) for f in csv_files]
merged_df = pd.concat(dfs, ignore_index=True)

# 4️⃣ 중복 제거 (같은 RECIPE_ID가 여러 카테고리에 있을 수 있음)
merged_df.drop_duplicates(subset=["RECIPE_ID"], inplace=True)

# 5️⃣ 최종 CSV 저장
output_name = "recipes_over100k_all.csv"
merged_df.to_csv(output_name, index=False, encoding="utf-8-sig")

print(f"\n✅ 병합 완료! 총 {len(merged_df)}개 레시피 → {output_name}")


📂 병합할 파일 목록:
 - recipes_over100k_국_탕.csv
 - recipes_over100k_기타.csv
 - recipes_over100k_메인반찬.csv
 - recipes_over100k_면_만두.csv
 - recipes_over100k_밑반찬.csv
 - recipes_over100k_밥_죽_떡.csv
 - recipes_over100k_샐러드.csv
 - recipes_over100k_스프.csv
 - recipes_over100k_양식.csv
 - recipes_over100k_찌개.csv
 - recipes_over100k_퓨전.csv

✅ 병합 완료! 총 1442개 레시피 → recipes_over100k_all.csv


In [19]:
import pandas as pd
import ast  # 문자열 딕셔너리 파싱용

# ✅ 분류 기준
INGREDIENT_KEYWORDS = {
    "소고기": "소고기류", "쇠고기": "소고기류", "양지": "소고기류",
    "돼지고기": "돼지고기류", "삼겹살": "돼지고기류", "목살": "돼지고기류",
    "닭": "닭고기류", "오리": "닭고기류",
    "새우": "해물류", "오징어": "해물류", "조개": "해물류",
    "굴": "해물류", "멸치": "해물류", "참치": "해물류",
    "채소": "채소류", "양파": "채소류", "당근": "채소류",
    "감자": "채소류", "버섯": "버섯류", "두부": "콩/견과류",
}

PRIORITY = [
    ("소고기", "소고기류"), ("쇠고기", "소고기류"), ("양지", "소고기류"),
    ("돼지고기", "돼지고기류"), ("삼겹살", "돼지고기류"),
    ("닭", "닭고기류"), ("오리", "닭고기류"),
    ("새우", "해물류"), ("오징어", "해물류"), ("조개", "해물류"),
    ("굴", "해물류"), ("멸치", "해물류"), ("참치", "해물류"),
]

# ✅ 1️⃣ CSV 불러오기 (병합된 파일 경로 입력)
df = pd.read_csv("recipes_over100k_all.csv")

# ✅ 2️⃣ IRDNT_CODE 분류 함수
def classify_ingredient(ingredient_text):
    try:
        ingredients = ast.literal_eval(ingredient_text.replace("'", '"'))  # 문자열 → dict
        names = ingredients.keys()
    except Exception:
        return "기타"

    # 1️⃣ 우선순위 그룹 먼저 확인
    for keyword, category in PRIORITY:
        if any(keyword in name for name in names):
            return category

    # 2️⃣ 일반 키워드 검색
    for name in names:
        for keyword, category in INGREDIENT_KEYWORDS.items():
            if keyword in name:
                return category

    # 3️⃣ 해당 없으면 기타
    return "기타"

# ✅ 3️⃣ IRDNT_CODE 생성
df["IRDNT_CODE"] = df["INGREDIENT_FULL"].apply(classify_ingredient)

# ✅ 4️⃣ 컬럼 순서 재정렬
cols = [
    "RECIPE_ID", "RECIPE_NM_KO", "SUMRY", "SERVINGS", "TY_NM",
    "COOKING_TIME", "LEVEL_NM", "IRDNT_CODE",
    "INGREDIENT_FULL", "STEP_TEXT", "STEP_TIP"
]
df = df[cols]

# ✅ 5️⃣ 저장
#df.to_csv("recipes_over100k_all_with_IRDNT.csv", index=False, encoding="utf-8-sig")

print(f"✅ IRDNT_CODE 추가 완료! 총 {len(df)}개 레시피 → recipes_over100k_all_with_IRDNT.csv")


✅ IRDNT_CODE 추가 완료! 총 1442개 레시피 → recipes_over100k_all_with_IRDNT.csv


In [4]:
# 파일 경로
import pandas as pd
df = pd.read_csv("recipes_over100k_all_with_IRDNT.csv", encoding='utf-8-sig')  # 한글 CSV는 utf-8

In [3]:
!pip install \
    selenium \
    beautifulsoup4 \
    pandas \
    lxml \
    webdriver-manager \
    numpy \
    requests \
    tqdm

  Using cached selenium-4.37.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached beautifulsoup4-4.14.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached pandas-2.3.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached lxml-6.0.2-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.6 kB)
  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached numpy-2.3.4-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached trio-0.31.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached websocket_client-1.9.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached so

In [36]:
print(df['LEVEL_NM'].unique())      # 난이도 이름 목록
print(df['IRDNT_CODE'].unique())    # 재료 코드 목록

['아무나' '초급' '중급' '고급']
['해물류' '기타' '소고기류' '닭고기류' '채소류' '콩/견과류' '버섯류' '돼지고기류']


In [21]:
df[df['LEVEL_NM'].isna()]


,RECIPE_ID,RECIPE_NM_KO,SUMRY,SERVINGS,TY_NM,COOKING_TIME,LEVEL_NM,IRDNT_CODE,INGREDIENT_FULL,STEP_TEXT,STEP_TIP
143,6830965,<요리>등촌칼국수 같은 버섯 매운탕 소고기 샤브샤브 만들기,NaN,NaN,국/탕,NaN,NaN,버섯류,"{'느타리버섯': '3줌', '미나리': '1단', '대파': '1대', '양파':...",[],[]
470,4984086,안동찜닭으로 친구들과 맥주파티~~,NaN,NaN,메인반찬,NaN,NaN,기타,{},[],[]
895,5412563,누구나 좋아하는 감자조림♩,NaN,NaN,밑반찬,NaN,NaN,기타,{},[],[]
939,5010998,초간단 시금치나물,NaN,NaN,밑반찬,NaN,NaN,기타,{},[],[]
1203,4274565,"영양만점 쇠고기 야채 주먹밥 (어린이집 소풍도시락, 스피드한 도시락)",NaN,NaN,밥/죽/떡,NaN,NaN,기타,{},[],[]
1264,5309525,콘샐러드 :: KFC 콘샐러드 따라잡기,NaN,NaN,샐러드,NaN,NaN,기타,{},[],[]
1322,6891796,참치두부짜글이찌개 밥도둑 인정,NaN,NaN,찌개,NaN,NaN,해물류,"{'두부': '1모', '양파': '1/2개', '참치': '작은거 1캔', '대파...",[],[]
1419,5803328,보글보글 뚝배기 참치김치찌개 / 쉬운 김치찌개,NaN,NaN,찌개,NaN,NaN,기타,{},[],[]


In [22]:
df['INGREDIENT_FULL'].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
1437    False
1438    False
1439    False
1440    False
1441    False
Name: INGREDIENT_FULL, Length: 1442, dtype: bool

In [23]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1442 entries, 0 to 1441
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RECIPE_ID        1442 non-null   int64  
 1   RECIPE_NM_KO     1442 non-null   object 
 2   SUMRY            1409 non-null   object 
 3   SERVINGS         1374 non-null   float64
 4   TY_NM            1442 non-null   object 
 5   COOKING_TIME     1392 non-null   float64
 6   LEVEL_NM         1434 non-null   object 
 7   IRDNT_CODE       1442 non-null   object 
 8   INGREDIENT_FULL  1442 non-null   object 
 9   STEP_TEXT        1442 non-null   object 
 10  STEP_TIP         1442 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 124.0+ KB


In [5]:
df.isnull().sum()

RECIPE_ID           0
RECIPE_NM_KO        0
SUMRY               0
SERVINGS           54
TY_NM               0
COOKING_TIME       36
LEVEL_NM            0
IRDNT_CODE          0
INGREDIENT_FULL     0
STEP_TEXT           0
STEP_TIP            0
dtype: int64

In [6]:
import numpy as np

df = df.replace(['[]', '{}', ''], np.nan)

In [26]:
df = df.dropna(subset=['STEP_TEXT'])


In [37]:
df.isnull().sum()

RECIPE_ID           0
RECIPE_NM_KO        0
SUMRY               0
SERVINGS           54
TY_NM               0
COOKING_TIME       36
LEVEL_NM            0
IRDNT_CODE          0
INGREDIENT_FULL     0
STEP_TEXT           0
STEP_TIP            0
dtype: int64

In [7]:
df.head( )

,RECIPE_ID,RECIPE_NM_KO,SUMRY,SERVINGS,TY_NM,COOKING_TIME,LEVEL_NM,IRDNT_CODE,INGREDIENT_FULL,STEP_TEXT,STEP_TIP
0,6847470,담백한 새우연두부달걀탕,새우연두부달걀탕,2.0,국/탕,15.0,아무나,해물류,"{'연두부': '1팩', '새우': '10-12마리', '달걀': '1개', '송송...","[""1. 우선 다시마물을 만들어요. 냄비에 물 붓고 다시마를 넣어 센 불에서 끓이다...","["""", """", """", """", """", """", """"]"
1,6902441,새우살로 만드는 얼큰한 새우탕 만들기,새우탕,2.0,국/탕,60.0,아무나,해물류,"{'냉동': '새우살 200g', '양파': '1/2개', '애호박': '1/2개'...","[""1. · 멸치 다시 육수를 끓여줍니다 ( 저는 집에 다시팩이 있어서 넣어줬습니다...","[""- 다시마 3장, 멸치 7~10개, 물 1L넣고 끓이다가 물이 끓으면 다시마만 ..."
2,6915735,깔끔고소한 아침국~~들깨무국,들깨무국,2.0,국/탕,15.0,아무나,기타,"{'채썬': '무 1컵', '대파': '1줄', '물': '3컵', '들깨가루': ...","[""1. 채썬 무를 냄비에 넣고, 소금1/2큰술,참기름1큰술을 넣고 볶아요."", ""...","["""", """", """", """", """", """"]"
3,6914749,감자어묵국 보들보들하게 즐기는 간단국요리,감자어묵국,4.0,국/탕,30.0,초급,해물류,"{'감자': '1개', '양파': '1/2개', '어묵': '100g', '대파':...","[""1. 물 7컵에 다시재료를 넣어주고 중약불에서 뭉근하게 끓여 육수를 내줍니다.""...","["""", ""기호에 따라 청양고추 반개 분량을 총총 썰어 넣어주셔도 좋습니다."", ""..."
4,6880695,소고기 배춧국 만들기 든든한 한끼로 제격!,소고기배추국,3.0,국/탕,30.0,아무나,소고기류,"{'배추잎': '5~6장', '소고기국거리': '200g', '다진마늘': '1/2...","[""1. 배추 반포기를 다?! 노노~~ 배추잎 5장 정도 뜯어서 칼로 먹기 좋게 썰...","["""", """", """", """", """", """", """", """", """", """", """"]"


In [8]:
df[df['SUMRY'].isna()]


,RECIPE_ID,RECIPE_NM_KO,SUMRY,SERVINGS,TY_NM,COOKING_TIME,LEVEL_NM,IRDNT_CODE,INGREDIENT_FULL,STEP_TEXT,STEP_TIP


In [9]:
df=df.drop(columns=['RECIPE_NM_KO'])

In [10]:
df = df.rename(columns={'SUMRY': 'RECIPE_NM_KO'})


In [11]:
df.head()

,RECIPE_ID,RECIPE_NM_KO,SERVINGS,TY_NM,COOKING_TIME,LEVEL_NM,IRDNT_CODE,INGREDIENT_FULL,STEP_TEXT,STEP_TIP
0,6847470,새우연두부달걀탕,2.0,국/탕,15.0,아무나,해물류,"{'연두부': '1팩', '새우': '10-12마리', '달걀': '1개', '송송...","[""1. 우선 다시마물을 만들어요. 냄비에 물 붓고 다시마를 넣어 센 불에서 끓이다...","["""", """", """", """", """", """", """"]"
1,6902441,새우탕,2.0,국/탕,60.0,아무나,해물류,"{'냉동': '새우살 200g', '양파': '1/2개', '애호박': '1/2개'...","[""1. · 멸치 다시 육수를 끓여줍니다 ( 저는 집에 다시팩이 있어서 넣어줬습니다...","[""- 다시마 3장, 멸치 7~10개, 물 1L넣고 끓이다가 물이 끓으면 다시마만 ..."
2,6915735,들깨무국,2.0,국/탕,15.0,아무나,기타,"{'채썬': '무 1컵', '대파': '1줄', '물': '3컵', '들깨가루': ...","[""1. 채썬 무를 냄비에 넣고, 소금1/2큰술,참기름1큰술을 넣고 볶아요."", ""...","["""", """", """", """", """", """"]"
3,6914749,감자어묵국,4.0,국/탕,30.0,초급,해물류,"{'감자': '1개', '양파': '1/2개', '어묵': '100g', '대파':...","[""1. 물 7컵에 다시재료를 넣어주고 중약불에서 뭉근하게 끓여 육수를 내줍니다.""...","["""", ""기호에 따라 청양고추 반개 분량을 총총 썰어 넣어주셔도 좋습니다."", ""..."
4,6880695,소고기배추국,3.0,국/탕,30.0,아무나,소고기류,"{'배추잎': '5~6장', '소고기국거리': '200g', '다진마늘': '1/2...","[""1. 배추 반포기를 다?! 노노~~ 배추잎 5장 정도 뜯어서 칼로 먹기 좋게 썰...","["""", """", """", """", """", """", """", """", """", """", """"]"


In [12]:
df.to_csv("recipes_over100k_all_with_IRDNT.csv", index=False, encoding="utf-8-sig")